In [1]:
import numpy as np
import DeepMIMO
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from scipy.io import loadmat, savemat 

## 1) Generate Dataset 

In [2]:
# Load the default parameters
parameters = DeepMIMO.default_params()
# Scenario O1_60 extracted at the dataset_folder
parameters['scenario'] = 'O1_60'
parameters['dataset_folder'] = r'C:\\Users\\emre.topcu\\Desktop\\ulakws\\scenarios\\O1_60' # Set DeepMIMO dataset folder that has O1_60

In [3]:
parameters['num_paths'] = 5
# User rows 1-100
parameters['user_row_first'] = 1000
parameters['user_row_last'] = 1010
# Activate only the first basestation
parameters['active_BS'] = np.array([1,5,8]) 

parameters['OFDM']['bandwidth'] = 0.5 # 50 MHz
parameters['OFDM']['subcarriers'] = 1024 # OFDM with 512 subcarriers
parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['enable_BS2BS'] = False

parameters['ue_antenna']['shape'] = np.array([1, 1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32, 8]) # ULA of 32 elements
parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'
parameters['ue_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [4]:
# Print the default parameters
for i,j in parameters.items():
    print(i,": ,", j)

dataset_folder : , C:\\Users\\emre.topcu\\Desktop\\ulakws\\scenarios\\O1_60
scenario : , O1_60
dynamic_settings : , {'first_scene': 1, 'last_scene': 1}
num_paths : , 5
active_BS : , [1 5 8]
user_row_first : , 1000
user_row_last : , 1010
row_subsampling : , 1
user_subsampling : , 1
bs_antenna : , {'shape': array([ 1, 32,  8]), 'spacing': 0.5, 'radiation_pattern': 'halfwave-dipole'}
ue_antenna : , {'shape': array([1, 1, 1]), 'spacing': 0.5, 'radiation_pattern': 'halfwave-dipole'}
enable_BS2BS : , False
OFDM_channels : , 1
OFDM : , {'subcarriers': 1024, 'subcarriers_limit': 64, 'subcarriers_sampling': 1, 'bandwidth': 0.5, 'RX_filter': 0}


In [5]:
# Generate dataset
dataset = DeepMIMO.generate_data(parameters)


Basestation 1

UE-BS Channels


Generating channels: 100%|██████████| 1991/1991 [00:06<00:00, 300.81it/s]



Basestation 5

UE-BS Channels


Generating channels: 100%|██████████| 1991/1991 [00:06<00:00, 286.28it/s]



Basestation 8

UE-BS Channels


Generating channels: 100%|██████████| 1991/1991 [00:04<00:00, 442.35it/s]


In [6]:
# Examination of the dataset
print("Datatype: ", type(dataset), "- Items contained: ", len(dataset))
print("Datatype: ", type(dataset[0]), "- Items contained: ", len(dataset[0].items()))
print("Key names: ", list(dataset[0].keys()))
print("Value names: ", list(dataset[0]['user']))

Datatype:  <class 'list'> - Items contained:  3
Datatype:  <class 'dict'> - Items contained:  3
Key names:  ['user', 'basestation', 'location']
Value names:  ['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel']


## 2) Codebook 

In [7]:
def beamforming_codebook(ant_shape = np.array([1, 32, 1]), oversampling_rate = np.array([1, 1, 1]), kd = 0.5):
    
    kd = 2 * np.pi * kd
    codebook_size = ant_shape * oversampling_rate
    
    vecs = []
    for dim in range(3):
        # Transpose
        ind = np.arange(ant_shape[dim]).reshape((-1, 1))
        codebook_ang = np.linspace(0, np.pi, codebook_size[dim], endpoint = False).reshape((1, -1))                                                                                                     
        vec = np.sqrt(1./ant_shape[dim]) * np.exp(-1j * kd * ind * np.cos(codebook_ang))
        vecs.append(vec)
        
    F = np.kron(vecs[2], np.kron(vecs[1], vecs[0]))
    
    return F

In [8]:
# Codebook values
F = beamforming_codebook(ant_shape = parameters['bs_antenna'][0]['shape'], oversampling_rate = np.array([1, 2, 1]), kd = parameters['bs_antenna'][0]['spacing'])

In [9]:
F.shape

(256, 512)

# 3) Parameters

In [10]:
num_OFDM = int(parameters['OFDM']['subcarriers_limit']/parameters['OFDM']['subcarriers_sampling'])
num_beams = F.shape[1]
num_bs = len(parameters['active_BS'])
num_ue = len(parameters['active_UE'])

In [11]:
print(num_OFDM)
print(num_beams)
print(num_bs)
print(num_ue)

64
512
3
1991


In [12]:
# Noise figure at the base station
NF = 5
# Channel estimation processing gain          
Process_Gain = 10
# System bandwidth in Hz
BW = parameters['OFDM']['bandwidth'] * 1e9
# Noise power in dB
noise_power_dB = -204 + 10*np.log10(BW/parameters['OFDM']['subcarriers']) + NF - Process_Gain
# Noise power
noise_power = 10**(.1*(noise_power_dB))

In [13]:
input_norm = np.zeros((num_bs, num_ue, num_OFDM), dtype=complex)
max_rates = np.zeros((num_bs, num_ue, num_beams))
print("Shape of input:", input_norm.shape)
print("Shape of max-rates:", max_rates.shape)
print("Number of UEs: ", len(dataset[0]['user']['channel']))
print("Shape of channel parameters: ", dataset[0]['user']['channel'][0].shape)

Shape of input: (3, 1991, 64)
Shape of max-rates: (3, 1991, 512)
Number of UEs:  1991
Shape of channel parameters:  (1, 256, 64)


In [14]:
# Each BS
for bs_idx in tqdm(range(num_bs), desc='Neural Network Input-Output Generation-BS', position=0, leave=True):
    # Each UE
    for ue_idx in tqdm(range(num_ue), desc='Neural Network Input-Output Generation-BS-%i'%bs_idx, position=0, leave=True):
        ch = dataset[bs_idx]['user']['channel'][ue_idx].squeeze()
        ch = ch + np.sqrt(noise_power) * (np.random.randn(*(ch.shape)) + 1j * np.random.randn(*(ch.shape)))
        input_norm[bs_idx, ue_idx, :] = ch[0, :]
        max_rates[bs_idx, ue_idx, :] = np.sum(np.log2(1 + np.abs(ch.T.conj() @ F)**2),  axis = 0)/num_OFDM

Neural Network Input-Output Generation-BS: 100%|██████████| 3/3 [00:46<00:00, 15.44s/it]


In [37]:
dataset[0]['user']['channel'].shape
#(number of RX antennas) x (number of TX antennas) x (number of OFDM subcarriers)

(1991, 1, 256, 64)

In [15]:
# Input reshape - normalize
input_norm = np.transpose(input_norm, axes=[1, 0, 2])
input_norm = input_norm.reshape((num_ue, -1))
input_norm /=  np.amax(np.abs(input_norm))

In [51]:
input_norm.shape

(1991, 192)

In [16]:
# Output reshape - normalize
max_rates_norm_factor = np.amax(max_rates, axis=2, keepdims=True)
# Do not normalize if all zeros
max_rates_norm_factor[max_rates_norm_factor== 0] = 1
max_rates /= max_rates_norm_factor
max_rates = np.transpose(max_rates, axes=[1, 0, 2])
max_rates = max_rates.reshape((num_ue, -1))

In [53]:
max_rates.shape

(1991, 1536)

In [17]:
if not os.path.exists('./DLCB_dataset'):
                      os.makedirs('DLCB_dataset')
savemat('./DLCB_dataset/DLCB_input.mat', {'DL_input': input_norm})
savemat('./DLCB_dataset/DLCB_output.mat', {'DL_output': max_rates})

# 4) Machine Learning

In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, regularizers
from datetime import datetime
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [41]:
now = datetime.now()
formatted_date = now.strftime('%Y-%m-%d/%H_%M_%S')
print(formatted_date)

2024-08-02/09_47_39


In [43]:
def train(model, x_train, y_train, x_test, y_test,
          EPOCHS, BATCH_SIZE, dr, lr, l2_reg,
          num_hidden_layers, nodes_per_layer,
          loss_fn, num_bs, n_beams, filepath):
    """
    Trains a list of neural network models, one for each base station.

    Parameters:
    - model: Function to create the model
    - x_train: Training input data
    - y_train: Training output data
    - x_test: Testing input data
    - y_test: Testing output data
    - EPOCHS: Number of epochs to train the models
    - BATCH_SIZE: Batch size for training
    - dr: Dropout rate
    - lr: Learning rate for the optimizer
    - l2_reg: L2 regularization factor (default is 0.01).
    - num_hidden_layers: Number of hidden layers in the model
    - nodes_per_layer: Number of nodes per hidden layer
    - loss_fn: Loss function to use for training
    - num_bs: Number of base stations
    - n_beams: Number of beams per base station
    - filepath: Path to save the best model during training

    Returns:
    - AP_models: List of trained models, one for each base station
    """
    
    # Determine the input shape by excluding the batch dimension
    input_shape = list(x_train.shape[1:])
    print("Training data shapes:")
    print(f"x_train shape: {x_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"x_test shape: {x_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    # Initialize an empty list to store the trained models for each base station
    AP_models = []
    
    # Iterate over each base station to create and train a model
    for bs_idx in range(num_bs):
        # Generate a unique identifier for the current base station
        idx_str = f'BS{bs_idx}'
        idx = bs_idx*n_beams
        
        # Create a new model for the current base station using the UlakNET
        model = UlakNET(input_shape, nodes_per_layer, num_hidden_layers, dr, n_beams, idx_str, l2_reg)
        
        # Compile the model with the specified learning rate
        optimizer = optimizers.Adam(learning_rate=lr)
        model.compile(loss=loss_fn, optimizer=optimizer)
        
        model.summary()
        
        # Train the model with the training data and validate with the test data
        model.fit(x_train, y_train[:, idx:idx + n_beams],
                    batch_size = BATCH_SIZE,
                    epochs = EPOCHS,
                    verbose = 2,
                    validation_data = (x_test, y_test[:,idx:idx + n_beams]),
                    callbacks = [
                        # Save the best model based on validation loss
                        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto'),
                        # Stop training early if validation loss does not improve
                        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
                    ])
        
        # Append the trained model to the list of models
        AP_models.append(model)  
    return AP_models

In [44]:
def UlakNET(input_shape, nodes_per_layer, num_hidden_layers, dr, n_beams, idx_str, l2_reg=0.01):
    """
    Creates and returns a Sequential neural network model.

    Parameters:
    - input_shape: Shape of the input data (excluding batch size).
    - nodes_per_layer: Number of nodes in each hidden layer.
    - num_hidden_layers: Number of hidden layers.
    - dr: Dropout rate.
    - n_beams: Number of output units (beams).
    - idx_str: Index of a specific BS (Base Station).
    - l2_reg: L2 regularization factor (default is 0.01).

    Returns:
    - model: Compiled Keras Sequential model.
    """
    # Initialize a Sequential model
    model = models.Sequential()

    # Input layer with Dropout
    model.add(layers.Dense(nodes_per_layer, activation='relu',
                    kernel_initializer='he_normal',
                    input_shape=input_shape,
                    kernel_regularizer=tf.keras.regularizers.l2(l2_reg))
                    )
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dr))
    
    model.add(layers.Flatten())
    
    # Hidden layers
    for _ in range(num_hidden_layers):
        model.add(layers.Dense(nodes_per_layer*4, activation='relu',
                            kernel_initializer='he_normal',
                            kernel_regularizer=tf.keras.regularizers.l2(l2_reg))
                            )
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dr))

    # Output layer
    model.add(layers.Dense(n_beams,
                           activation='linear',
                           kernel_initializer='he_normal',
                           name=f"dense_{idx_str}_output"))
    return model

In [45]:
# Reading input and output sets generated from MATLAB
input_set_file=loadmat('DLCB_dataset/DLCB_input.mat')
output_set_file=loadmat('DLCB_dataset/DLCB_output.mat')

input_set=input_set_file['DL_input']
output_set=output_set_file['DL_output']

In [47]:
# Parameter initialization
num_user_tot=input_set.shape[0]
print("Total number of users:", num_user_tot)
n_DL_size=[0.01, .05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7]
cnt=0

Total number of users: 1991


In [49]:
for DL_size_ratio in n_DL_size:
    
    print("For DL size ratio:", DL_size_ratio)
    cnt += 1
    DL_size = int(num_user_tot * DL_size_ratio)
    print("Initializing...")
    
    # Train/Test Split
    np.random.seed(42)
    num_train = int(DL_size * 0.8)
    num_test = int(num_user_tot * 0.2)
    
    # Randomly select train and test indices
    train_index = np.random.choice(range(0,num_user_tot), size=num_train, replace=False)
    rem_index = set(range(0,num_user_tot))-set(train_index)
    test_index= list(set(np.random.choice(list(rem_index), size=num_test, replace=False)))
    print("--------------------")
    print("DL size: ", DL_size)
    print("train_size: ", len(train_index))
    print("rem_size: ", len(rem_index))
    print("test_size: ", len(test_index))
    print("--------------------")
   
    # Prepare training and testing data
    x_train = np.real(input_set[train_index])
    x_test = np.real(input_set[test_index])
    y_train = output_set[train_index]
    y_test = output_set[test_index]
    
    print("Setting Learning Parameters...")
    # Learning model parameters
    epochs = 100
    batch_size = 64
    dr = 0.2
    lr = 0.01
    l2_reg = 0.0005
    num_hidden_layers = 10
    nodes_per_layer = x_train.shape[1]
    loss_fn = 'mean_squared_error'
    filepath = f"./models/model_{formatted_date}.weights.h5"
    
    # Model training
    print("Starting Model Training...")
    AP_models = train(
        UlakNET, x_train, y_train, x_test, y_test,
        epochs, batch_size, dr, lr, l2_reg,
        num_hidden_layers, nodes_per_layer,
        loss_fn, num_bs, num_beams, filepath
    )
    
    print("Evaluating Model...")
    DL_Result = {}
    
    for idx in range(0,num_bs,1): 
        beams_predicted=AP_models[idx].predict(x_test, batch_size=10, verbose=0)
    
        DL_Result['TX'+str(idx+1)+'Pred_Beams'] = beams_predicted
        DL_Result['TX'+str(idx+1)+'Opt_Beams'] = y_test[:,idx*num_beams:(idx+1)*num_beams]

    DL_Result['user_index']=test_index
    
    
    if not os.path.exists('./DLCB_code_output'):
                          os.makedirs('DLCB_code_output')
    savemat('DLCB_code_output/DL_Result'+str(cnt)+'.mat',DL_Result)
    print("Iteration completed: ", cnt, "/", len(n_DL_size))
print("Training/Evaluation session finished!")

For DL size ratio: 0.01
Initializing...
--------------------
DL size:  19
train_size:  15
rem_size:  1976
test_size:  398
--------------------
Setting Learning Parameters...
Starting Model Training...
Training data shapes:
x_train shape: (15, 192)
y_train shape: (15, 1536)
x_test shape: (398, 192)
y_test shape: (398, 1536)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_33 (Dense)                │ (None, 192)            │        37,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 192)            │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 768)            │       148,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 768)            │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 768)            │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 768)            │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 768)            │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 768)            │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 768)            │         3,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,925,824 (22.61 MB)

 Trainable params: 5,910,080 (22.55 MB)

...

 Total params: 5,925,824 (22.61 MB)

 Trainable params: 5,910,080 (22.55 MB)

 Non-trainable params: 15,744 (61.50 KB)

Epoch 1/100
1/1 - 21s - 21s/step - loss: 10.4023 - val_loss: 9.4168
Epoch 2/100
1/1 - 0s - 338ms/step - loss: 10.5760 - val_loss: 26.1447
Epoch 3/100
1/1 - 0s - 306ms/step - loss: 10.9050 - val_loss: 141.4769
Epoch 4/100
1/1 - 0s - 375ms/step - loss: 11.2442 - val_loss: 991.7221
Epoch 5/100
1/1 - 0s - 345ms/step - loss: 11.5166 - val_loss: 6437.5039
Epoch 6/100
1/1 - 0s - 352ms/step - loss: 11.6086 - val_loss: 22256.5957
Evaluating Model...


KeyboardInterrupt: 

# 5) Read Results

In [ ]:
import glob
import re 

In [ ]:
output_text = """
Epoch 1/100
69/69 - 16s - 239ms/step - loss: 8.0320 - val_loss: 2.2982
Epoch 2/100
69/69 - 7s - 102ms/step - loss: 1.1148 - val_loss: 0.3673
Epoch 3/100
69/69 - 7s - 105ms/step - loss: 0.3192 - val_loss: 0.1198
Epoch 4/100
69/69 - 7s - 102ms/step - loss: 0.1101 - val_loss: 0.0600
Epoch 5/100
69/69 - 7s - 102ms/step - loss: 0.0630 - val_loss: 0.0371
Epoch 6/100
69/69 - 7s - 104ms/step - loss: 0.0408 - val_loss: 0.0232
Epoch 7/100
69/69 - 7s - 104ms/step - loss: 0.0227 - val_loss: 0.0172
Epoch 8/100
69/69 - 7s - 106ms/step - loss: 0.0150 - val_loss: 0.0120
Epoch 9/100
69/69 - 7s - 103ms/step - loss: 0.0166 - val_loss: 0.0064
Epoch 10/100
69/69 - 7s - 103ms/step - loss: 0.0064 - val_loss: 0.0040
Epoch 11/100
69/69 - 7s - 108ms/step - loss: 0.0033 - val_loss: 0.0029
Epoch 12/100
69/69 - 7s - 101ms/step - loss: 0.0030 - val_loss: 0.0031
Epoch 13/100
69/69 - 10s - 142ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 14/100
69/69 - 7s - 96ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 15/100
69/69 - 6s - 92ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 16/100
69/69 - 6s - 93ms/step - loss: 0.0033 - val_loss: 0.0031
    """

In [ ]:
def parse_training_output(output_text):
    """
    Parses the training output to extract epoch numbers, training loss, and validation loss.

    Parameters:
    - output_text: A string containing the training output log.

    Returns:
    - epochs: A list of epoch numbers.
    - train_loss: A list of training loss values.
    - val_loss: A list of validation loss values.
    """
    # Regular expressions to extract relevant information
    epoch_pattern = re.compile(r'Epoch (\d+)/\d+')
    loss_pattern = re.compile(r'loss: ([\d.]+)')
    val_loss_pattern = re.compile(r'val_loss: ([\d.]+)')

    # Find corresponding values
    epoch_matches = re.findall(epoch_pattern, output_text)
    loss_matches = re.findall(loss_pattern, output_text)
    val_loss_matches = re.findall(val_loss_pattern, output_text)

    # Convert extracted data to appropriate data types
    epochs = [int(match) for match in epoch_matches]
    losses = [float(loss) for loss in loss_matches]
    val_losses = [float(val_loss) for val_loss in val_loss_matches]

    # Since the regular expressions find all matches, the loss list will include both train and val losses.
    # We need to separate them correctly.
    train_losses = losses[::2]  # Every other item starting from the first
    val_losses = val_losses  # Already just the validation losses

    return epochs, train_losses, val_losses

In [ ]:
def plot_loss(epochs, train_loss, val_loss):
    """
    Plots the training and validation loss over epochs.

    Parameters:
    - epochs: List of epoch numbers.
    - train_loss: List of training loss values.
    - val_loss: List of validation loss values.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Parse the output to get epochs, training loss, and validation loss
epochs, train_loss, val_loss = parse_training_output(output_text)
print("Epochs:", epochs)
print("Training Loss:", train_loss)
print("Validation Loss:", val_loss)
# Plot the losses
plot_loss(epochs, train_loss, val_loss)

In [ ]:
file_list = sorted(glob.glob('DLCB_code_output/DL_Result*'), key=lambda x: int(re.findall(r'\d+', x)[0]))
num_files = len(file_list)

user_index = []
pred_beams = []
opt_beams = []
for file in tqdm(file_list, desc='Reading DL results'):
    matfile = loadmat(file)
    l1 = []
    l2 = []
    for idx in range(num_bs):
        if idx ==2 or idx ==3:
            continue
        l1.append(matfile['TX'+str(idx+1)+'Pred_Beams'])
        l2.append(matfile['TX'+str(idx+1)+'Opt_Beams'])
        
    pred_beams.append(l1)
    opt_beams.append(l2)
    user_index.append(matfile['user_index'])

In [ ]:
Pn = -204 + 10*np.log10(BW) # Noise power in dB
SNR = 10**(.1*(0-Pn))

ach_rate_DL = np.zeros(num_files)
ach_rate_opt = np.zeros(num_files)

eff_rate = np.zeros(num_files)
opt_rate = np.zeros(num_files)
for file_idx in tqdm(np.arange(num_files), desc = 'Calculating results'):
    user_index_file = user_index[file_idx].flatten()
    for ue_idx in range(len(user_index_file)):
        eff_ch = []
        opt_ch = []
        for bs_idx in range(2):
            if file_idx == 0: # Random BF - 0 Samples
                pred_beam_idx = np.random.randint(num_beams)
            else:
                pred_beam_idx = np.argmax(pred_beams[file_idx][bs_idx][ue_idx])
            opt_beam_idx = np.argmax(opt_beams[file_idx][bs_idx][ue_idx])
            ch_single_bs = dataset[bs_idx]['user']['channel'][user_index_file[ue_idx]].squeeze()
            eff_ch_single_pred = ch_single_bs.T.conj() @ F[:, pred_beam_idx]
            opt_ch_single_pred = ch_single_bs.T.conj() @ F[:, opt_beam_idx]
            eff_ch.append(eff_ch_single_pred)
            opt_ch.append(opt_ch_single_pred)
        eff_ch = np.array(eff_ch)
        opt_ch = np.array(opt_ch)
        eff_rate[file_idx] += np.sum(np.log2(1 + SNR * np.abs(np.diag(eff_ch.conj().T @ eff_ch))))
        opt_rate[file_idx] += np.sum(np.log2(1 + SNR * np.abs(np.diag(opt_ch.conj().T @ opt_ch))))
    eff_rate[file_idx] /= len(user_index_file)*num_OFDM
    opt_rate[file_idx] /= len(user_index_file)*num_OFDM


In [ ]:
# % Eff achievable rate calculations
theta_user=(102/parameters['bs_antenna'][0]['shape'][1])*np.pi/180
alpha=60*np.pi/180
distance_user=10
Tc_const=(distance_user*theta_user)/(2*np.sin(alpha)) # ms
Tt=10*1e-6; # ms

v_mph=50
v=v_mph*1000*1.6/3600 # m/s
Tc=Tc_const/v

overhead_opt=1-(num_beams*Tt)/Tc # overhead of beam training
overhead_DL=1-Tt/Tc # overhead of proposed DL method


# 6) Plots

In [ ]:
DL_size_array=np.arange(0, 2.5*(num_files), 2.5);

fig, ax = plt.subplots()
plt.plot(DL_size_array, opt_rate, '--k', label = 'Genie-aided Coordinated Beamforming')
plt.plot(DL_size_array, eff_rate*overhead_DL, '-bo', label = 'Deep Learning Coordinated Beamforming')
plt.plot(DL_size_array, opt_rate*overhead_opt, '-rs', label = 'Baseline Coordinated Beamforming')
plt.ylim([0, 6])
plt.xlim([0, 22])
plt.minorticks_on()
plt.grid()
plt.xlabel('Deep Learning Dataset Size (Thousand Samples)')
plt.ylabel('Achievable Rate (bps/Hz)')
plt.legend()
plt.savefig('result.png')
